In [1]:
import re
import pandas as pd

# load gold standard

In [2]:
gold_standard_df = pd.read_csv('information_retrieval/IR_theme_gold_standard_paragraphs.tsv',sep="\t", index_col=None)  
gold_standard_df = gold_standard_df.fillna("")

In [3]:
gold_standard_df

,Paper,Study design,Target population,Financial detail/costs,Person-level outcomes
0,#2598,This study used a sequential explanatory equal...,family- and center-based child care programs. ...,Five of the eight center directors not affilia...,
1,#17192,The evaluation of the SIB-CPC project is using...,Children were included in the intervention coh...,,"Of those 325 children, 59% (58.77%) were consi..."
2,#17247,bringing together the findings from interviews...,Indigenous Asháninka people of the Ene River i...,"OUTCOME FUNDS (USD) $110,000 // PAYMENT SCHEDU...",Results per outcome metric: 1) 75% of target a...
3,#17284,"We show, through three ECE case studies, how U...","3,500 students in Salt Lake City // 2,618 new ...","In the SIB contract, success payments are expe...","In the first cohort, 595 students attended pre..."
4,#17725,IDinsight conducted a three-year randomized co...,,,Outcome 1: Learning gains\nBy the end of Year ...
5,#17755,The focus of the evaluation is the DIBs fundin...,"more than 300,000 primary school aged children...",Up to CHF 26.09 million of outcome payments wi...,


In [4]:
from collections import defaultdict
gold_standard = defaultdict(dict)
for i, row in gold_standard_df.iterrows():
    paper = row["Paper"]
    sd = [sample.strip() for sample in row["Study design"].split("//")]
    tp = [sample.strip() for sample in row["Target population"].split("//")]
    fd = [sample.strip() for sample in row["Financial detail/costs"].split("//")]
    po = [sample.strip() for sample in row["Person-level outcomes"].split("//")]
    gold_standard["Study Design"][paper] = sd
    gold_standard["Target Population"][paper] = tp
    gold_standard["Financial detail and costs"][paper] = fd
    gold_standard["Person-level outcomes"][paper] = po

# load model results from Json

In [5]:
import json
with open('./28 Nov 2022_Papers _ variables for the new tests/model1.json', 'r') as f:
    data = json.load(f)
for q in data['#2598']:
    print(q)

Study Design
Target Population
Financial detail and costs
Personal-level outcomes
Financial detail and costs(filtered)


### convert json to dataframe for the evaluation

In [8]:
qa = {}
question_type = "Financial detail and costs(filtered)" # define question type based on the question type name from json
for paper in data:
    qa[paper] = []
for paper in data:
    for q in data[paper][question_type]:
        qa[paper].append("\n".join(["Top 1: " + s for s in data[paper][question_type][q]]))
model1_df = pd.DataFrame(data=qa, index=list(data["#2598"][question_type].keys()))
model1_df = (model1_df.T)
model1_df['Paper'] = model1_df.index
model1_df.head()

,What are the costs of the contract?,How much is paid for outcomes?,What are the outcomes payments?,What is the total contract value?,What is the price per outcome?,outcomes payment price contract value contract cap rate card incentive payment costs savings,outcomes payment,price,contract value,contract cap,rate card,incentive payment,costs,savings,Paper
#2598,Top 1: 1.2. Tiered child care reimbursement sy...,Top 1: 1.2. Tiered child care reimbursement sy...,"Top 1: below, only three center directors repo...",Top 1: 1.009** (0.002) 1.013** (0.003) 1.017**...,Top 1: 3.1.2. Regression results Model 1 is a ...,Top 1: Maryland is a good case study due to it...,Top 1: 3.2.3. EXCELS payments If center direct...,Top 1: Although the CCDF program encourages st...,Top 1: 1.009** (0.002) 1.013** (0.003) 1.017**...,Top 1: 1.009** (0.002) 1.013** (0.003) 1.017**...,Top 1: Although the CCDF program encourages st...,Top 1: The ﬁrst stage of research consisted of...,Top 1: Although the CCDF program encourages st...,Top 1: 1.009** (0.002) 1.013** (0.003) 1.017**...,#2598
#17247,Top 1: Impact bonds may be more expensive than...,"Top 1: With an impact bond outcome fund, a rat...",Top 1: Payments from the outcome funder to the...,Top 1: CAPITAL COMMITMENT IN IMPACT BONDS IN L...,"Top 1: With an impact bond outcome fund, a rat...","Top 1: With an impact bond outcome fund, a rat...",Top 1: Payments from the outcome funder to the...,Top 1: 12 | IMPACT BONDS IN DEVELOPING COUNTRI...,Top 1: CAPITAL COMMITMENT IN IMPACT BONDS IN L...,Top 1: CONTRACT LENGTH IN IMPACT BONDS IN LOW-...,"Top 1: With an impact bond outcome fund, a rat...",Top 1: Another important consideration is to i...,Top 1: Determining the level of payments can b...,Top 1: While the basic structure of impact bon...,#17247
#17284,Top 1: Despite details in available documentat...,Top 1: Goldman Sachs and the J.B. and M.K. Pri...,Top 1: Goldman Sachs and the J.B. and M.K. Pri...,"Top 1: In Utah, the SIB contract is not public...",Top 1: Intervention Clientele South Carolina N...,Top 1: Intervention Clientele South Carolina N...,Top 1: Goldman Sachs and the J.B. and M.K. Pri...,Top 1: Despite details in available documentat...,Top 1: Despite details in available documentat...,Top 1: Despite details in available documentat...,Top 1: Utah’s SIB designers were concerned wit...,"Top 1: Goldman Sachs, the J.B. and M.K. Pritzk...",Top 1: Despite details in available documentat...,Top 1: The SIB’s designers clearly considered ...,#17284
#17755,Top 1: Contract management costs attributable ...,Top 1: These ranged from USD 650k to USD 6.4m ...,Top 1: Up to USD 2.8 million of outcomes payme...,Top 1: Implementation Contract management cost...,Top 1: These ranged from USD 650k to USD 6.4m ...,Top 1: Potential approaches which could bring ...,Top 1: Up to USD 2.8 million of outcomes payme...,Top 1: Investment vehicle costs varied dependi...,Top 1: and activities Supplier demonstrates ho...,Top 1: Contract management costs attributable ...,Top 1: Identifying metrics and structuring pay...,Top 1: Up to USD 2.8 million of outcomes payme...,Top 1: Investment vehicle costs varied dependi...,Top 1: Identification of individuals who live ...,#17755
#17192,Top 1: 2015 2016 2017 2018 2019 2020 2021 2022...,Top 1: In addition to these impact outcome que...,Top 1: Payments based on Kindergarten Readines...,Top 1: 2015 2016 2017 2018 2019 2020 2021 2022...,Top 1: Investigating Highly Unexpected Outcome...,Top 1: Child-Parent Center Evaluation: Report ...,Top 1: Payments based on Kindergarten Readines...,Top 1: 2015 2016 2017 2018 2019 2020 2021 2022...,Top 1: 2015 2016 2017 2018 2019 2020 2021 2022...,Top 1: Child-Parent Center Evaluation: Report ...,Top 1: 3rd 3rd 3rd 2nd 2nd 2nd 5th 5th 4th 4th...,Top 1: 2015 2016 2017 2018 2019 2020 2021 2022...,Top 1: 2015 2016 2017 2018 2019 2020 2021 2022...,Top 1: The base cohort sizes are based on the ...,#17192


# if not json then load excel

In [9]:
question_type = "Financial detail and costsFilte"  # define question type based on the question type name from excel
model1_df = pd.read_excel('./28 Nov 2022_Papers _ variables for the new tests/model2.xlsx', index_col=None, sheet_name=question_type)  
model1_df = model1_df.fillna("")
model1_df.head()

,Paper,What are the costs of the contract?,How much is paid for outcomes?,What are the outcomes payments?,What is the total contract value?,What is the price per outcome?,Outcomes payment; price; contract value; contract cap; rate card; incentive payment; costs; savings.
0,#2598,Top 1: 3. Results 3.1. Quantitative results 3...,Top 1: 3. Results 3.1. Quantitative results 3...,Top 1: A director of a disconﬁrming center th...,Top 1: 3. Results 3.1. Quantitative results 3...,Top 1: 3. Results 3.1. Quantitative results 3...,Top 1: reported that each of two locations ha...
1,#17247,Top 1: 7. CREATE GLOBAL INVESTMENT FUNDS Whil...,"Top 1: With an impact bond outcome fund, a ra...","Top 1: With an impact bond outcome fund, a ra...",Top 1: 21 | EARLY LEARNINGS FROM THE FIELD TA...,"Top 1: With an impact bond outcome fund, a ra...","Top 1: With an impact bond outcome fund, a ra..."
2,#17284,Top 1: Despite details in available documenta...,Top 1: Goldman Sachs and the J.B. and M.K. Pr...,Top 1: Goldman Sachs and the J.B. and M.K. Pr...,Top 1: Goldman Sachs and the J.B. and M.K. Pr...,Top 1: Goldman Sachs and the J.B. and M.K. Pr...,Top 1: Intervention Clientele South Carolina ...
3,#17755,Top 1: Investment vehicle costs varied depend...,Top 1: Return to investors The maximum paymen...,"Top 1: outcome payment of USD 0–USD 412,000 t...",Top 1: 100% of outputs answer all evaluation ...,Top 1: Return to investors The maximum paymen...,"Top 1: outcome payment of USD 0–USD 412,000 t..."
4,#17192,Top 1:  Two-thirds of the children (68%) wer...,Top 1:  Two-thirds of the children (68%) wer...,Top 1:  Two-thirds of the children (68%) wer...,Top 1:  Two-thirds of the children (68%) wer...,Top 1:  Two-thirds of the children (68%) wer...,Top 1:  Two-thirds of the children (68%) wer...


In [17]:
question_type_gold = "Financial detail and costs" # define the question type name from gold standard. ()

,What are the costs of the contract?,How much is paid for outcomes?,What are the outcomes payments?,What is the total contract value?,What is the price per outcome?,outcomes payment price contract value contract cap rate card incentive payment costs savings,outcomes payment,price,contract value,contract cap,rate card,incentive payment,costs,savings,Paper
#2598,Top 1: 1.2. Tiered child care reimbursement sy...,Top 1: 1.2. Tiered child care reimbursement sy...,"Top 1: below, only three center directors repo...",Top 1: 1.009** (0.002) 1.013** (0.003) 1.017**...,Top 1: 3.1.2. Regression results Model 1 is a ...,Top 1: Maryland is a good case study due to it...,Top 1: 3.2.3. EXCELS payments If center direct...,Top 1: Although the CCDF program encourages st...,Top 1: 1.009** (0.002) 1.013** (0.003) 1.017**...,Top 1: 1.009** (0.002) 1.013** (0.003) 1.017**...,Top 1: Although the CCDF program encourages st...,Top 1: The ﬁrst stage of research consisted of...,Top 1: Although the CCDF program encourages st...,Top 1: 1.009** (0.002) 1.013** (0.003) 1.017**...,#2598
#17247,Top 1: Impact bonds may be more expensive than...,"Top 1: With an impact bond outcome fund, a rat...",Top 1: Payments from the outcome funder to the...,Top 1: CAPITAL COMMITMENT IN IMPACT BONDS IN L...,"Top 1: With an impact bond outcome fund, a rat...","Top 1: With an impact bond outcome fund, a rat...",Top 1: Payments from the outcome funder to the...,Top 1: 12 | IMPACT BONDS IN DEVELOPING COUNTRI...,Top 1: CAPITAL COMMITMENT IN IMPACT BONDS IN L...,Top 1: CONTRACT LENGTH IN IMPACT BONDS IN LOW-...,"Top 1: With an impact bond outcome fund, a rat...",Top 1: Another important consideration is to i...,Top 1: Determining the level of payments can b...,Top 1: While the basic structure of impact bon...,#17247
#17284,Top 1: Despite details in available documentat...,Top 1: Goldman Sachs and the J.B. and M.K. Pri...,Top 1: Goldman Sachs and the J.B. and M.K. Pri...,"Top 1: In Utah, the SIB contract is not public...",Top 1: Intervention Clientele South Carolina N...,Top 1: Intervention Clientele South Carolina N...,Top 1: Goldman Sachs and the J.B. and M.K. Pri...,Top 1: Despite details in available documentat...,Top 1: Despite details in available documentat...,Top 1: Despite details in available documentat...,Top 1: Utah’s SIB designers were concerned wit...,"Top 1: Goldman Sachs, the J.B. and M.K. Pritzk...",Top 1: Despite details in available documentat...,Top 1: The SIB’s designers clearly considered ...,#17284
#17755,Top 1: Contract management costs attributable ...,Top 1: These ranged from USD 650k to USD 6.4m ...,Top 1: Up to USD 2.8 million of outcomes payme...,Top 1: Implementation Contract management cost...,Top 1: These ranged from USD 650k to USD 6.4m ...,Top 1: Potential approaches which could bring ...,Top 1: Up to USD 2.8 million of outcomes payme...,Top 1: Investment vehicle costs varied dependi...,Top 1: and activities Supplier demonstrates ho...,Top 1: Contract management costs attributable ...,Top 1: Identifying metrics and structuring pay...,Top 1: Up to USD 2.8 million of outcomes payme...,Top 1: Investment vehicle costs varied dependi...,Top 1: Identification of individuals who live ...,#17755
#17192,Top 1: 2015 2016 2017 2018 2019 2020 2021 2022...,Top 1: In addition to these impact outcome que...,Top 1: Payments based on Kindergarten Readines...,Top 1: 2015 2016 2017 2018 2019 2020 2021 2022...,Top 1: Investigating Highly Unexpected Outcome...,Top 1: Child-Parent Center Evaluation: Report ...,Top 1: Payments based on Kindergarten Readines...,Top 1: 2015 2016 2017 2018 2019 2020 2021 2022...,Top 1: 2015 2016 2017 2018 2019 2020 2021 2022...,Top 1: Child-Parent Center Evaluation: Report ...,Top 1: 3rd 3rd 3rd 2nd 2nd 2nd 5th 5th 4th 4th...,Top 1: 2015 2016 2017 2018 2019 2020 2021 2022...,Top 1: 2015 2016 2017 2018 2019 2020 2021 2022...,Top 1: The base cohort sizes are based on the ...,#17192
#17725,Top 1: Educate Girls Development Impact Bond F...,Top 

In [18]:
from collections import Counter
def isOverlap(input_text, gold_text):
    temp1 = input_text.lower().strip()
    temp2 = gold_text.lower().strip()
    count1 = Counter(temp1.split())
    count2 = Counter(temp2.split())
    
    overlap1 = 0
    for k in count1:
        if k in count2 and count1[k] >= count2[k]:
            overlap1 += count2[k]
        elif k in count2 and count1[k] < count2[k]:
            overlap1 += count1[k]
    overlap1 = overlap1/len(temp1.split())
    if overlap1 >= 0.5:
        return True
    else:
        overlap2 = 0
        for k in count2:
            if k in count1 and count2[k] <= count1[k]:
                overlap2 += count2[k]
            elif k in count1 and count2[k] > count1[k]:
                overlap2 +=  count1[k]
        overlap2 = overlap2/len(temp2.split())
        if overlap2 >= 1.0:
            return True
        else:
            return False
        
def isOverlap2(input_text, gold_text):
    A = input_text.lower().strip().split()
    B = gold_text.lower().strip().split()
    n = len(A)
    m = len(B)
 
    # Auxiliary dp[][] array
    dp = [[0 for i in range(m + 1)] for i in range(n + 1)]
 
    # Updating the dp[][] table
    # in Bottom Up approach
    for i in range(n - 1, -1, -1):
        for j in range(m - 1, -1, -1):
 
            # If A[i] is equal to B[i]
            # then dp[j][i] = dp[j + 1][i + 1] + 1
            if A[i] == B[j]:
                dp[i][j] = dp[i + 1][j + 1] + 1
    maxm = 0
    # Find maximum of all the values
    # in dp[][] array to get the
    # maximum length
    for i in dp:
        for j in i:
            # Update the length
            maxm = max(maxm, j)
 

    if maxm/n >= 0.5:
        return True
    elif maxm/m >= 0.5:
        return True
    else:
        return False
        
isOverlap2("this is a test", "this is a test right")

True

In [19]:
top = 20
precision = {}
for i, row in model1_df.iterrows():
    paper = row["Paper"]
    gold = gold_standard[question_type_gold][paper]
    precision[paper] = []
    if gold == [""]:
        continue
    for k in model1_df:
        if k == "Paper":
            continue
        results = re.split("top\s+\d+\s*\:", row[k].lower().strip())[1:]
#         assert len(results) == 25
        c = 0
        for sample in results[:top]:
            for g in gold:
                if isOverlap2(sample, g):
                    c += 1
                    break
        precision[paper].append(c/top)
        print(paper, k, "Precison@20:", c/top)
        print()

#2598 What are the costs of the contract? Precison@20: 0.05

#2598 How much is paid for outcomes? Precison@20: 0.0

#2598 What are the outcomes payments? Precison@20: 0.0

#2598 What is the total contract value? Precison@20: 0.0

#2598 What is the price per outcome? Precison@20: 0.0

#2598 outcomes payment price contract value contract cap rate card incentive payment costs savings Precison@20: 0.0

#2598 outcomes payment Precison@20: 0.0

#2598 price Precison@20: 0.0

#2598 contract value Precison@20: 0.0

#2598 contract cap Precison@20: 0.0

#2598 rate card Precison@20: 0.0

#2598 incentive payment Precison@20: 0.0

#2598 costs Precison@20: 0.0

#2598 savings Precison@20: 0.0

#17247 What are the costs of the contract? Precison@20: 0.0

#17247 How much is paid for outcomes? Precison@20: 0.1

#17247 What are the outcomes payments? Precison@20: 0.1

#17247 What is the total contract value? Precison@20: 0.05

#17247 What is the price per outcome? Precison@20: 0.0

#17247 outcomes payment

In [20]:
top = 20
recall = {}
for i, row in model1_df.iterrows():
    paper = row["Paper"]
    gold = gold_standard[question_type][paper]
    recall[paper] = []
    if gold == [""]:
        continue
    for k in model1_df:
        if k == "Paper":
            continue
        results = re.split("top\s+\d+\s*\:", row[k].lower().strip())[1:]
#         print(paper)
#         assert len(results) == 25
        c = 0
        for g in gold:
            for sample in results[:top]:
                if isOverlap2(sample, g):
                    c += 1
                    break
        recall[paper].append(c/len(gold))
        print(paper, k, "Recall@20:", c/len(gold))
        print()

#2598 What are the costs of the contract? Recall@20: 0.5

#2598 How much is paid for outcomes? Recall@20: 0.0

#2598 What are the outcomes payments? Recall@20: 0.0

#2598 What is the total contract value? Recall@20: 0.0

#2598 What is the price per outcome? Recall@20: 0.0

#2598 outcomes payment price contract value contract cap rate card incentive payment costs savings Recall@20: 0.0

#2598 outcomes payment Recall@20: 0.0

#2598 price Recall@20: 0.0

#2598 contract value Recall@20: 0.0

#2598 contract cap Recall@20: 0.0

#2598 rate card Recall@20: 0.0

#2598 incentive payment Recall@20: 0.0

#2598 costs Recall@20: 0.0

#2598 savings Recall@20: 0.0

#17247 What are the costs of the contract? Recall@20: 0.0

#17247 How much is paid for outcomes? Recall@20: 0.10344827586206896

#17247 What are the outcomes payments? Recall@20: 0.10344827586206896

#17247 What is the total contract value? Recall@20: 0.10344827586206896

#17247 What is the price per outcome? Recall@20: 0.0

#17247 outcomes

In [21]:
# for copy paste to excel spreedsheet.
for k in precision:
    for i, v in enumerate(precision[k]):
        print("{:.2f}/{:.2f}".format(v, recall[k][i]), end=" ")
    print()

0.05/0.50 0.00/0.00 0.00/0.00 0.00/0.00 0.00/0.00 0.00/0.00 0.00/0.00 0.00/0.00 0.00/0.00 0.00/0.00 0.00/0.00 0.00/0.00 0.00/0.00 0.00/0.00 
0.00/0.00 0.10/0.10 0.10/0.10 0.05/0.10 0.00/0.00 0.05/0.10 0.10/0.10 0.00/0.00 0.05/0.10 0.05/0.10 0.00/0.00 0.05/0.10 0.00/0.00 0.15/0.10 
0.05/0.50 0.05/0.50 0.05/0.50 0.05/0.50 0.05/0.50 0.10/1.00 0.05/0.50 0.05/0.50 0.05/0.50 0.05/0.50 0.05/0.50 0.05/0.50 0.05/0.50 0.00/0.00 
0.25/0.40 0.25/0.40 0.15/0.20 0.20/0.33 0.25/0.40 0.20/0.40 0.15/0.20 0.05/0.07 0.10/0.13 0.15/0.27 0.00/0.00 0.15/0.20 0.10/0.13 0.05/0.07 


